In [1]:
# conda activate covid19
import numpy as np
import pandas as pd
np.__version__, pd.__version__

('1.19.2', '1.2.2')

1) Load the raw csv file into a pandas dataframe

In [2]:
# https://data.ontario.ca/dataset/confirmed-positive-cases-of-covid-19-in-ontario
csvName = 'data/conposcovidloc-2021-02-23'
csvName = 'data/conposcovidloc-2021-03-07'
rawDf = pd.read_csv(csvName + '.csv', 
                  header=0, 
                  index_col='Row_ID', 
                  parse_dates=['Accurate_Episode_Date','Case_Reported_Date','Test_Reported_Date','Specimen_Date'])
rawDf.head()

,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
Row_ID,,,,,,,,,,,,,,,,,
1,2020-05-04,2020-05-13,2020-05-13,2020-05-07,50s,FEMALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
2,2020-04-08,2020-04-17,2020-04-17,2020-04-10,50s,MALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
3,2020-03-21,2020-04-04,2020-04-04,2020-04-02,50s,FEMALE,NO KNOWN EPI LINK,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
4,2020-05-13,2020-05-17,2020-05-17,2020-05-15,60s,MALE,OB,Resolved,Yes,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
5,2020-06-12,2020-06-15,2020-06-15,2020-06-14,20s,MALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893


In [3]:
rawDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 308296 entries, 1 to 308296
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Accurate_Episode_Date      308296 non-null  datetime64[ns]
 1   Case_Reported_Date         308296 non-null  datetime64[ns]
 2   Test_Reported_Date         297240 non-null  datetime64[ns]
 3   Specimen_Date              306378 non-null  datetime64[ns]
 4   Age_Group                  308296 non-null  object        
 5   Client_Gender              308296 non-null  object        
 6   Case_AcquisitionInfo       308296 non-null  object        
 7   Outcome1                   308296 non-null  object        
 8   Outbreak_Related           68461 non-null   object        
 9   Reporting_PHU_ID           308296 non-null  int64         
 10  Reporting_PHU              308296 non-null  object        
 11  Reporting_PHU_Address      308296 non-null  object  

In [4]:
# https://www.geeksforgeeks.org/creating-a-sqlite-database-from-csv-with-python/

createDb = True

if createDb:

    # Import required libraries 
    import sqlite3 

    # Connect to SQLite database 
    conn = sqlite3.connect(r'conposcovidloc.db') 

    # Write the data to a sqlite table 
    rawDf.to_sql(csvName, conn, if_exists='replace', index=True) 

    # Close connection to SQLite database 
    conn.close() 

2) Extract the Public Health Unit data into a csv file

In [5]:
phuIndexColumn = 'Reporting_PHU_ID'
phuInfoColumns = ['Reporting_PHU', 'Reporting_PHU_Address','Reporting_PHU_City','Reporting_PHU_Postal_Code','Reporting_PHU_Website','Reporting_PHU_Latitude','Reporting_PHU_Longitude']
phuAllColumns =  [phuIndexColumn] + phuInfoColumns
phuDf = rawDf[phuAllColumns].set_index(phuIndexColumn).drop_duplicates()
phuDf.to_csv('PublicHealthUnits.csv')
phuDf.head()

,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
Reporting_PHU_ID,,,,,,,
2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
2236,Halton Region Health Department,1151 Bronte Road,Oakville,L6M 3Ll,www.halton.ca/For-Residents/Public-Health/,43.413997,-79.744796
2270,York Region Public Health Services,17250 Yonge Street,Newmarket,L3Y 6Z1,www.york.ca/wps/portal/yorkhome/health/,44.048023,-79.480239
2230,Durham Region Health Department,605 Rossland Road East,Whitby,L1N 0B2,www.durham.ca/en/health-and-wellness/health-an...,43.898605,-78.940341
3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358


In [6]:
phuDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 2253 to 2243
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Reporting_PHU              34 non-null     object 
 1   Reporting_PHU_Address      34 non-null     object 
 2   Reporting_PHU_City         34 non-null     object 
 3   Reporting_PHU_Postal_Code  34 non-null     object 
 4   Reporting_PHU_Website      34 non-null     object 
 5   Reporting_PHU_Latitude     34 non-null     float64
 6   Reporting_PHU_Longitude    34 non-null     float64
dtypes: float64(2), object(5)
memory usage: 2.1+ KB


3) Drop the Public Health Unit Information columns from the raw data frame into a new caseDf dataframe.

In [7]:
caseDf = rawDf.drop(phuInfoColumns, axis = 1)
caseDf['Outbreak_Related'] = caseDf['Outbreak_Related'].fillna('No')
caseDf.head()

,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID
Row_ID,,,,,,,,,,
1,2020-05-04,2020-05-13,2020-05-13,2020-05-07,50s,FEMALE,CC,Resolved,No,2253
2,2020-04-08,2020-04-17,2020-04-17,2020-04-10,50s,MALE,CC,Resolved,No,2253
3,2020-03-21,2020-04-04,2020-04-04,2020-04-02,50s,FEMALE,NO KNOWN EPI LINK,Resolved,No,2253
4,2020-05-13,2020-05-17,2020-05-17,2020-05-15,60s,MALE,OB,Resolved,Yes,2253
5,2020-06-12,2020-06-15,2020-06-15,2020-06-14,20s,MALE,CC,Resolved,No,2253


In [8]:
caseDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 308296 entries, 1 to 308296
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Accurate_Episode_Date  308296 non-null  datetime64[ns]
 1   Case_Reported_Date     308296 non-null  datetime64[ns]
 2   Test_Reported_Date     297240 non-null  datetime64[ns]
 3   Specimen_Date          306378 non-null  datetime64[ns]
 4   Age_Group              308296 non-null  object        
 5   Client_Gender          308296 non-null  object        
 6   Case_AcquisitionInfo   308296 non-null  object        
 7   Outcome1               308296 non-null  object        
 8   Outbreak_Related       308296 non-null  object        
 9   Reporting_PHU_ID       308296 non-null  int64         
dtypes: datetime64[ns](4), int64(1), object(5)
memory usage: 25.9+ MB


In [9]:
for column in enumerate(['Age_Group','Client_Gender','Case_AcquisitionInfo','Outcome1','Outbreak_Related']):
    print(column[1], '=>', caseDf[column[1]].unique().tolist())

Age_Group => ['50s', '60s', '20s', '80s', '40s', '70s', '30s', '90+', '<20', 'UNKNOWN']
Client_Gender => ['FEMALE', 'MALE', 'UNSPECIFIED', 'GENDER DIVERSE']
Case_AcquisitionInfo => ['CC', 'NO KNOWN EPI LINK', 'OB', 'TRAVEL', 'MISSING INFORMATION', 'UNSPECIFIED EPI LINK']
Outcome1 => ['Resolved', 'Fatal', 'Not Resolved']
Outbreak_Related => ['No', 'Yes']


7) Save the caseDf into a csv file

In [10]:
caseDf.to_csv('PositiveCases.csv')